In [39]:
# following https://stackoverflow.com/questions/45492703/google-drive-api-oauth-and-service-account

In [7]:
#  ! pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib google-cloud-translate

In [93]:
# ! pip install transformers==3.1
# ! pip3 install --upgrade tensorflow-gpu

In [6]:
from google.cloud import translate

In [16]:
import json
from tqdm.auto import tqdm

In [54]:
from transformers import pipeline
import time
import re
from nltk.tokenize import sent_tokenize
import pandas as pd
import numpy as np

In [17]:
tqdm.pandas()

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [10]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'data/service.json'

In [3]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

SCOPES = ['https://www.googleapis.com/auth/documents.readonly']
SCOPES += ['https://www.googleapis.com/auth/drive.metadata.readonly']
SERVICE_ACCOUNT_FILE = 'data/service.json'

creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

In [7]:
def get_doc_content(doc_id):
    service = build('docs', 'v1', credentials=creds)

    # Retrieve the documents contents from the Docs service.
    document = service.documents().get(documentId=doc_id).execute()

    paragraphs = [paragraph['paragraph'] for paragraph in document['body']['content'] if 'paragraph' in paragraph]
    elements = [element['textRun']['content'] for p in paragraphs for element in p['elements']]
    
    return ''.join(elements)



In [ ]:
# get file
# get date per file
# translate file
# get file sentiment

In [51]:
def translate_text(text="Hello, world!", project_id="swift-clarity-183805"):

    client = translate.TranslationServiceClient()
    location = "global"
    parent = f"projects/{project_id}/locations/{location}"
    
    response = client.translate_text(
        request={
            "parent": parent,
            "contents": [text],
            "mime_type": "text/plain",
            "source_language_code": "he",
            "target_language_code": "en-US",
        }
    )

    for translation in response.translations:
        yield("Translated text: {}".format(translation.translated_text))



In [31]:
def split_to_sentences(data):
    sentences = [s for d in data for s in d.split('\n')]
    sentences = [s.strip() for s in sentences]
    sentences = [re.sub('&#39;', '\'', s) for s in sentences]
    sentences = [re.sub('…', '...', s) for s in sentences]
    sentences = [s for line in sentences for s in sent_tokenize(line) if s]
    new_sentences = []
    for s in sentences:
        if len(s)>=512:
            new_sentences += s.split(',')
        else:
            new_sentences.append(s)
    return new_sentences

In [12]:
service = build('drive', 'v3', credentials=creds)

In [73]:
file_list = service.files().list(pageSize=1000).execute()

In [74]:
files = file_list['files']

In [75]:
files[0]

{'kind': 'drive#file',
 'id': '1MYr8DigaeVwMBVripFXGWF6Gvi7N0-QDXTp4vGV_A50',
 'name': 'Untitled document',
 'mimeType': 'application/vnd.google-apps.document'}

In [76]:
for f in tqdm(files):
    f.update(service.files().get(fileId=f['id'], fields="createdTime").execute())

In [ ]:
sentiment_pipeline = pipeline("sentiment-analysis")

In [52]:
def get_sentiment(data):
    data = [s for p in data for s in split_to_sentences(p)]
    sentiments = sentiment_pipeline(data)
    score = sum(s['score'] if s['label']=='POSITIVE' else -s['score'] for s in sentiments)/len(sentiments)
    return score

In [78]:
files_df = pd.DataFrame(files)

In [80]:
files_df.to_csv('data/thoughts/files_metadata.csv')

In [63]:
# ! mkdir data/thoughts/ids

In [ ]:
for file in tqdm(files):
    if file['mimeType']!='application/vnd.google-apps.document':
        continue
    if os.path.exists(f"data/thoughts/ids/{file['id']}.txt"):
        continue
    content = get_doc_content(file['id'])
    translation = translate_text(content)
    with open(f'data/thoughts/ids/{file["id"]}.txt', 'w') as f:
        f.write('\n'.join(translation))